MNIST is a dataset of handwritten digits, containing 60,000 training images and 10,000 testing images. All images are in grayscale, each with a size of 28x28. In this section, we will build a Feed Forward Neural Network to classify an input image of handrwitten digits from 0-9.

In [1]:
import torch
import torch.nn as nn # in pytorch github we have torch folder and inside we have nn folder. all the files inside nn are taken here
import torchvision.datasets as datasets  # datasets used in neural networks
import torchvision.transforms as transforms # pytorch- vision -trasforms folder in pytorch github

In [2]:
input_size = 784        #Number of input neurons (image pixels)
hidden_size = 400       #Number of hidden neurons (i/p+o/p)/2
out_size = 10           #Number of classes (0-9) 
epochs = 10            #How many times we pass our entire dataset into our network 
batch_size = 100        #Input size of the data during one iteration (100 images fed into neural network for every iteration)
learning_rate = 0.001   #How fast we are learning

In [3]:
train_dataset = datasets.MNIST(root='./data',  # path where images will be saved
                           train=True,         # train dataset
                           transform=transforms.ToTensor(),  # what we do on dataset. we transform the images to tensors
                           download=True)                   # downloads the dataset

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...


C:\Users\Vikee\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,      # dataloader for train data
                                          batch_size=batch_size,
                                          shuffle=True)                 

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,              # dataloader for test data
                                          batch_size=batch_size,
                                          shuffle=False)     # false because not interested in order, just testing here

## Building the Network

![alt text](bb.png "Title")

In [18]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)   # nn.Linear to create a layer with i/p and o/p size #First Layer                           
        self.fc2 = nn.Linear(hidden_size, hidden_size)      #Second Layer Activation hidden layer as i./p and another hidden layer as o/p
        self.fc3 = nn.Linear(hidden_size, out_size)   # third layer with input as hidden layer and o/p as output
        self.relu = nn.ReLU()


    def forward(self, x):    # forward propagation
        out = self.fc1(x)
        out = self.relu(out)  # use relu activation function for hidden layers
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [19]:
# Create the network (an object of the Net class)

net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()   # To switch between cpu and gpu. if true, it will be passed to gpu.
if CUDA:
    net = net.cuda()  # gpu

In [20]:
#The loss function. The Cross Entropy loss comes along with Softmax in pytorch. Therefore, no need to specify Softmax as well

criterion = nn.CrossEntropyLoss()


In [21]:
# Adam Optimizer

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [22]:
net.parameters()  

<generator object Module.parameters at 0x0000025ACEC266C8>

In [23]:
list(net.parameters()) # array for each layers

[Parameter containing:
 tensor([[ 0.0309, -0.0238,  0.0055,  ..., -0.0340,  0.0255, -0.0173],
         [ 0.0255,  0.0165,  0.0118,  ..., -0.0221, -0.0303,  0.0159],
         [ 0.0012,  0.0156, -0.0320,  ..., -0.0328, -0.0084,  0.0068],
         ...,
         [ 0.0141,  0.0110, -0.0281,  ..., -0.0303, -0.0007, -0.0067],
         [ 0.0357, -0.0046,  0.0107,  ...,  0.0235,  0.0254, -0.0259],
         [-0.0206, -0.0324,  0.0205,  ..., -0.0247,  0.0105, -0.0279]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 2.8564e-02,  1.5610e-02,  1.6629e-02,  3.7557e-03, -1.7458e-02,
         -3.3423e-02, -1.2102e-02,  3.0721e-02, -2.0534e-02,  4.2123e-04,
          2.3171e-02, -6.5271e-03,  2.4582e-02, -2.0775e-02,  2.2620e-02,
         -3.9294e-03,  2.8632e-02, -5.3002e-03, -2.1069e-02, -1.4115e-02,
         -1.0154e-02, -1.2128e-02,  3.4386e-03,  7.9938e-03, -3.4941e-04,
          2.2082e-02,  1.8171e-02, -2.8237e-03, -2.3478e-02,  4.1604e-04,
         -6.5536e-03, -

In [24]:
net.parameters # to see the layers. all layers have bias by default

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=10, bias=True)
  (relu): ReLU()
)>

## Visualise the train loader and reshaping the image

In [26]:
for i,(images,labels) in enumerate(train_loader):
    print(images.size())
    images=images.view(-1,784)   # TO RESIZE
    print(images.size()) 

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([

torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([

torch.Size([100, 1, 28, 28]) 

here 1 (channels)indicates gray scale image (if 3, RGB image) , 100 indicates the batch size, 28 is height and 28 is width of image

after reshaping it becomes 100 times 784


## Train the Network

there is no direct training method in pytorch. we need to write it step by step


In [34]:
correct_train = 0
total_train = 0
running_loss = 0
for epoch in range(epochs):
    
  
    for i, (images, labels) in enumerate(train_loader):   
        #Flatten the image from size (batch,1,28,28) --> (100,1,28,28) where 1 represents the number of channels (grayscale-->1),
        # to size (100,784) and wrap it in a variable
        images = images.view(-1, 28*28) 
        
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
            
        # w <-- w - lr * gradient -- we need to clear this gradient buffer. we don't want to accumulate gradients
        optimizer.zero_grad() 
              
        # Feed Forward.
        outputs = net(images)   
        
        _, predicted = torch.max(outputs.data, 1)   # torch.max(data,1) find maximum value for each row and puts it in an array
                                                    # here we use this predicted instead of output as in lsast application
        total_train +=labels.size(0)   
        if CUDA:
            correct_train += (predicted.cpu() == labels.cpu()).sum()
        else:
            correct_train += (predicted == labels).sum() 
            
        
        
        loss = criterion(outputs, labels)                 # Difference between the actual and predicted (loss function)
        running_loss += loss.item()
        
        loss.backward()                                   # Backpropagation
        optimizer.step()                                  # Update the weights
        
    print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%'.format
          (epoch+1, epochs, running_loss//len(train_loader), (100*correct_train//total_train)))
print("DONE TRAINING!")

Epoch [1/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [2/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [3/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [4/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [5/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [6/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [7/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [8/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [9/10], Training Loss: 0.000, Training Accuracy: 99.000%
Epoch [10/10], Training Loss: 0.000, Training Accuracy: 99.000%
DONE TRAINING!


## Test the Network

for testing no weight,loss calculations needed
ony forward feed

In [44]:
correct = 0
for images, labels in test_loader:
    if CUDA:
            images = images.cuda()
            labels = labels.cuda()
    images = images.view(-1, 28*28)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()

print('Test Accuracy : %d %%'%(100 * correct / len(test_dataset)))
    
        

Test Accuracy : 98 %
